In [1]:
import psycopg2
from psycopg2 import OperationalError
import pandas as pd
import json

df = pd.read_csv("C:/Users/valen/OneDrive/Documentos/workshop_1/dataset/candidates.csv", sep=";")

C:\Users\valen\AppData\Local\Temp\ipykernel_9516\3681093721.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [7]:
#Creating the database
def create_database(db_name):
    with open('database.json', 'r') as config_file:
        config = json.load(config_file)

    conn = psycopg2.connect(
        host=config['host'],
        user=config['user'],
        password=config['password']
    )
    conn.autocommit = True

    cursor = conn.cursor()

    cursor.execute(f"DROP DATABASE IF EXISTS {db_name};")
    cursor.execute(f"CREATE DATABASE {db_name};")

    cursor.close()
    conn.close()
    print(f"the database '{db_name}' has been sucessfully created.")


La base de datos 'workshop2' ha sido creada con éxito.


In [3]:
def connect_to_db():
    db_conn = None
    try:
        with open('db_config.json', 'r') as config_file:
            db_settings = json.load(config_file)

        db_conn = psycopg2.connect(
            host='localhost',
            user=db_settings['user'],
            password=db_settings['password'],
            dbname=db_settings['database']
        )
        print('Connection to the database was successful')
    except psycopg2.DatabaseError as db_error:
        print('Failed to connect to the database:', db_error)
    return db_conn

connect_to_db()

Connection to the database was successful


<connection object at 0x000002448B592130; dsn: 'user=postgres password=xxx dbname=workshop3 host=localhost', closed: 0>

In [10]:
def setup_candidates_table():
    table_creation_sql = '''
        CREATE TABLE IF NOT EXISTS applicants (
            CandidateID SERIAL PRIMARY KEY,
            first_name VARCHAR(255) NOT NULL,
            last_name VARCHAR(255) NOT NULL,
            email_address VARCHAR(255) NOT NULL,
            date_of_application DATE NOT NULL,
            country_of_origin VARCHAR(255) NOT NULL,
            experience_years INT NOT NULL,
            level_of_seniority VARCHAR(255) NOT NULL,
            tech_stack VARCHAR(255) NOT NULL,
            code_challenge_result SMALLINT NOT NULL,
            interview_score SMALLINT NOT NULL,
            hired_status BOOLEAN NOT NULL
        );
    '''
    db_connection = None
    try:
        db_connection = connect_to_db()
        db_cursor = db_connection.cursor()
        db_cursor.execute(table_creation_sql)
        db_cursor.close()
        db_connection.commit()
        print('The "applicants" table has been successfully created.')
    except (Exception, psycopg2.DatabaseError) as db_error:
        print('Error while creating the table:', db_error)
    finally:
        if db_connection is not None:
            db_connection.close()

setup_candidates_table()

Connection to the database was successful
The "applicants" table has been successfully created.


In [12]:
def insertar_datos(df):
    conn = connect_to_db()
    if conn is None:
        print("No se pudo establecer la conexión con la base de datos.")
        return

    cursor = conn.cursor()
    query = """
    INSERT INTO applicants (first_name, last_name, email_address, date_of_application, country_of_origin, experience_years, level_of_seniority, tech_stack, code_challenge_result, interview_score, hired_status)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    try:
        for index, row in df.iterrows():
            is_hired = row['Code Challenge Score'] >= 7 and row['Technical Interview Score'] >= 7

            # Crear una tupla con los datos a insertar
            data = (row["First Name"], row["Last Name"], row["Email"], row["Application Date"], row["Country"],
                    row["YOE"], row["Seniority"], row["Technology"], row["Code Challenge Score"], row["Technical Interview Score"], is_hired)

            cursor.execute(query, data)

        conn.commit()
        print("Datos insertados exitosamente")
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error al insertar los datos:", error)
        conn.rollback()
    finally:
        cursor.close()

insertar_datos(df)

Connection to the database was successful
Datos insertados exitosamente
